In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek\


from itertools import combinations

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 10:24:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/20 10:24:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
def create_new_spark_context(appName):
    return SparkSession.builder.appName(appName).config("spark.driver.memory", "32g").config("spark.driver.maxResultSize", "0")\
    .master("local[*]").getOrCreate()

spark_session = create_new_spark_context("Q4")
sc = spark_session.sparkContext

In [ ]:
schema = StructType([
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True),
        StructField("ORIGINE_CAR_KEY",StringType(),True),
        StructField("FINAL_CAR_KEY",StringType(),True),
        StructField("CHECK_STATUS_KEY", IntegerType(), True),
        StructField("COMPANY_ID", StringType(), True),
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [ ]:
df = spark_session.read.csv('/content/drive/MyDrive/MDA/EX2/Sample_Traffic.csv', header=True, schema=schema)
df.show(1)

In [51]:
traffic_rdd = df.rdd.map(lambda x: ((x[3], x[6].date()), x[0])).groupByKey().mapValues(list)
traffic = traffic_rdd.collect()

In [54]:
traffic_rdd[:5]

ERROR:root:KeyboardInterrupt while sending command.               (0 + 12) / 12]
Traceback (most recent call last):
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/nkazemi/opt/anaconda3/envs/myEnv/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

: 

In [ ]:
support_threshold  = 1000

In [41]:
count_single_item_frequncy_rdd = traffic_rdd.flatMap(lambda x : x[1]).map(lambda x: (x , 1)).reduceByKey(lambda x, y: x + y)

In [42]:
count_single_item_frequncy_rdd.take(10)

[(631776, 40),
 (22010052, 56),
 (22010088, 271),
 (631356, 91),
 (22010040, 269),
 (900132, 145),
 (100700820, 257),
 (100701144, 66),
 (631368, 151),
 (631608, 46)]

In [43]:
single_frequent_path = count_single_item_frequncy_rdd.filter(lambda x: x[1] > support_threshold).keys().collect()

In [44]:
single_frequent_path = set(single_frequent_path)

In [45]:
test = traffic_rdd.filter(lambda x: 900101 in x[1]).take(1)
test

[(('22486571', datetime.date(2021, 6, 1)), [900101])]

In [46]:
def filter_only_pairs_candidate(x):
    res = []
    pairs_in_x = itertools.combinations(set(x[1]) , 2)
    for pair in pairs_in_x:
        if(all(item in single_frequent_path for item in pair)):
            res.append((pair , 1))
    return res


def filter_only_k_member_candidates(x , k , previous_frequents_candidates):
    res = []
    k_members_in_x = itertools.combinations(set(x[1]) , k)
    for k_member in k_members_in_x:
        supper_set_k_member = itertools.combinations(k_member , k-1)
        if(all(item in previous_frequents_candidates for item in supper_set_k_member)):
            res.append((k_member , 1))
    return res

In [47]:
filter_only_pairs_candidate(test[0])

[]

In [30]:
pairs_count_rdd = traffic_rdd.map(lambda x: filter_only_pairs_candidate(x))

In [31]:
pairs_count_rdd.take(5)

[[], [], [], [], []]

In [36]:
pairs_count_rdd = pairs_count_rdd.flatMap(lambda x: x)

In [ ]:
pairs_count_rdd.take(5)

In [38]:
pairs_count_rdd.repartition(48)

MapPartitionsRDD[48] at coalesce at NativeMethodAccessorImpl.java:0

In [39]:
all_frequent_pairs = pairs_count_rdd.reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] > support_threshold).keys().collect()

In [109]:
all_frequent_pairs

[(900226, 100700804),
 (100700804, 900246),
 (900102, 900212),
 (900212, 900246),
 (22010117, 900241),
 (900276, 900246),
 (22009971, 900215),
 (900233, 100700845),
 (900233, 900149),
 (206602, 900240),
 (212802, 22009972),
 (900268, 900222),
 (900266, 900276),
 (900266, 900240),
 (900101, 900265),
 (209103, 119),
 (631367, 631763),
 (900225, 100700853),
 (100700804, 900202),
 (22010119, 900191),
 (900107, 22009971),
 (900265, 900149),
 (100700864, 900142),
 (100701130, 900244),
 (900225, 631829),
 (900237, 631829),
 (900135, 119),
 (900259, 22010119),
 (900101, 900273),
 (900202, 900212),
 (631829, 900217),
 (900265, 100700845),
 (900245, 22009977),
 (100700841, 100700853),
 (900139, 900207),
 (22010061, 100700845),
 (900269, 631829),
 (900269, 900217),
 (900272, 900202),
 (100700868, 900246),
 (900164, 900222),
 (114, 100700824),
 (22009972, 103002),
 (900101, 101301),
 (100700864, 900242),
 (900246, 631356),
 (100700834, 100700804),
 (900217, 631357),
 (900101, 202601),
 (631795, 90

In [66]:
def get_k_member_candidates(frequent_tuples , k):
    k_member_candidates = []
    elements = set(itertools.chain.from_iterable(frequent_tuples))
    all_k_pairs = itertools.combinations(elements , k)
    for k_pair in all_k_pairs:
        supper_sets = itertools.combinations(set(k_pair) , k-1)
        if (all(item in frequent_tuples for item in supper_sets)):
            k_member_candidates.append(k_pair)
    return k_member_candidates

In [111]:
# def filter_only_k_member_candidates(x , k , k_member_candidates):
#     res = []
#     k_members_in_x = itertools.combinations(set(x[1]) , k)
#     for k_member in k_members_in_x:
#         if(k_member in k_member_candidates):
#             res.append((k_member , 1))
#     return res

In [126]:
def filter_only_k_member_candidates(x , k , previous_frequents_candidates):
    res = []
    k_members_in_x = itertools.combinations(set(x[1]) , k)
    for k_member in k_members_in_x:
        supper_set_k_member = itertools.combinations(k_member , k-1)
        if(all(item in previous_frequents_candidates for item in supper_set_k_member)):
            res.append((k_member , 1))
    return res

In [127]:
def filter_only_pairs_candidate(x):
    res = []
    pairs_in_x = itertools.combinations(set(x[1]) , 2)
    for pair in pairs_in_x:
        if(all(item in single_frequent_path for item in pair)):
            res.append((pair , 1))
    return res

In [128]:
def get_k_frequents(k , traffic_rdd:SparkContext , previous_frequnet_tuples):
    k_count_rdd = traffic_rdd.map(lambda x: filter_only_k_member_candidates(x , k , previous_frequnet_tuples)).flatMap(lambda x: x)
    k_member_frequents = k_count_rdd.reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] > support_threshold).keys().collect()
    return k_member_frequents

In [129]:
get_k_frequents(3 , traffic_rdd , all_frequent_pairs)

[(900139, 900268, 100700826),
 (206602, 900234, 100700845),
 (900102, 100701130, 631357),
 (900259, 100700841, 900269),
 (900101, 900268, 900212),
 (900225, 900244, 900212),
 (100700804, 900265, 900266),
 (900233, 205802, 900234),
 (900142, 100700853, 900212),
 (900225, 900269, 900207),
 (900164, 900107, 100700824),
 (900139, 900212, 100700826),
 (900212, 900244, 22009977),
 (900259, 900101, 100700841),
 (100700841, 900244, 900212),
 (900256, 142, 900215),
 (900273, 900212, 900244),
 (100700866, 100700868, 900155),
 (100700866, 900155, 900222),
 (100700839, 900142, 900244),
 (900244, 900212, 100700853),
 (212802, 205802, 900234),
 (900101, 900273, 900212),
 (900225, 900101, 900268),
 (22010118, 205802, 900234),
 (212802, 22010118, 900234),
 (900135, 900240, 117),
 (900142, 900212, 900244),
 (202601, 100700841, 900236),
 (900142, 900212, 900152),
 (100700841, 900236, 900255),
 (900227, 900268, 900269),
 (231, 900107, 900236),
 (900259, 900268, 900269),
 (900268, 900269, 22009977),
 (900

In [70]:
triple_frequnets = get_k_frequents(3 , traffic_rdd , all_frequent_pairs)

[(900142, 900212, 900244), (900142, 900212, 100700853)]

In [130]:
def run_apriori_algorithm(traffic_rdd:SparkContext , support_threshold):
    count_single_item_frequncy_rdd = traffic_rdd.flatMap(lambda x : x[1]).map(lambda x: (x , 1)).reduceByKey(lambda x, y: x + y)
    single_frequent_path = set(count_single_item_frequncy_rdd.filter(lambda x: x[1] > support_threshold).keys().collect())
    pairs_count_rdd = traffic_rdd.map(lambda x: filter_only_pairs_candidate(x)).flatMap(lambda x: x)
    all_frequent_pairs = pairs_count_rdd.reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] > support_threshold).keys().collect()
    final_list = []
    final_list.append(single_frequent_path)
    check_list_size = all_frequent_pairs
    k = 3
    while(len(check_list_size) != 0):
        final_list.append(check_list_size)
        k_frequents = get_k_frequents(k , traffic_rdd , check_list_size)
        check_list_size = k_frequents
        k = k + 1
    return final_list

In [131]:
support_threshold = 1000

In [132]:
final_list = run_apriori_algorithm(traffic_rdd , support_threshold)

In [133]:
final_list

[{100,
  102,
  103,
  104,
  110,
  112,
  113,
  114,
  115,
  117,
  118,
  119,
  123,
  128,
  135,
  137,
  140,
  142,
  143,
  144,
  145,
  149,
  150,
  153,
  155,
  156,
  157,
  160,
  163,
  168,
  169,
  175,
  231,
  232,
  101301,
  103001,
  103002,
  107301,
  200202,
  200301,
  200402,
  200502,
  200601,
  200902,
  201801,
  202101,
  202601,
  202901,
  203001,
  203101,
  203301,
  203401,
  203501,
  203701,
  203902,
  204401,
  204901,
  204902,
  205201,
  205202,
  205501,
  205801,
  205802,
  206601,
  206602,
  206701,
  207101,
  207501,
  208601,
  208602,
  209101,
  209102,
  209103,
  210602,
  211301,
  211401,
  211701,
  211901,
  212001,
  212002,
  212101,
  212501,
  212601,
  212701,
  212802,
  213401,
  213402,
  230101,
  230102,
  230103,
  230106,
  230107,
  230108,
  230201,
  230204,
  631346,
  631349,
  631352,
  631354,
  631355,
  631356,
  631357,
  631359,
  631360,
  631361,
  631362,
  631363,
  631366,
  631367,
  631368,
  

In [135]:
final_list[3]

[(900142, 900212, 100700853, 900249),
 (231, 100700841, 900236, 900255),
 (900256, 900234, 142, 900215),
 (900225, 900259, 900268, 900269),
 (900142, 900244, 900212, 100700853),
 (900142, 900212, 100700853, 900152),
 (900142, 900212, 900244, 100700853),
 (900101, 231, 100700841, 900236),
 (900256, 22010118, 900234, 900215),
 (100700834, 100700804, 900265, 209103)]

Now we run SON algorithm for this we will divide rdd into 4 splits and then run a priori on each

In [152]:
indexed_traffic_rdd = traffic_rdd.zipWithIndex()
indexed_traffic_rdd.take(5)

[((('40682798', datetime.date(2021, 6, 1)), [206602]), 0),
 ((('12335890', datetime.date(2021, 6, 1)), [205202]), 1),
 ((('36298029', datetime.date(2021, 6, 1)), [230101, 114]), 2),
 ((('20428752', datetime.date(2021, 6, 1)), [230102]), 3),
 ((('8517992', datetime.date(2021, 6, 1)), [205201, 205202]), 4)]

In [153]:
son_rdd_part1 = indexed_traffic_rdd.filter(lambda x: x[1]%3 == 0).map(lambda x: x[0])
son_rdd_part2 = indexed_traffic_rdd.filter(lambda x: x[1]%3 == 1).map(lambda x: x[0])
son_rdd_part3 = indexed_traffic_rdd.filter(lambda x: x[1]%3 == 2).map(lambda x: x[0])

In [164]:
support_threshold = support_threshold/3

In [165]:
support_threshold

333.3333333333333

In [166]:
res1 = run_apriori_algorithm(son_rdd_part1 , support_threshold)
res2 = run_apriori_algorithm(son_rdd_part2 , support_threshold)
res3 = run_apriori_algorithm(son_rdd_part3 , support_threshold)

In [167]:
res1[3]

[(900142, 900212, 100700853, 900249),
 (900256, 900234, 142, 900215),
 (231, 100700841, 900236, 900255),
 (900225, 900259, 900268, 900269),
 (22010087, 22010088, 22010094, 22010095),
 (900142, 900244, 900212, 100700853),
 (900142, 900212, 100700853, 900152),
 (900256, 22010118, 900234, 900215),
 (900101, 231, 100700841, 900236),
 (100700839, 900142, 900212, 900244),
 (900142, 900212, 900244, 100700853),
 (100700834, 100700804, 900265, 209103)]

In [168]:
res2[3]

[(900256, 900234, 142, 900215),
 (231, 100700841, 900236, 900255),
 (900225, 900259, 900268, 900269),
 (900142, 900244, 900212, 100700853),
 (900142, 900212, 900244, 100700853),
 (900101, 231, 100700841, 900236),
 (900256, 22010118, 900234, 900215),
 (100700834, 100700804, 900265, 209103)]

In [169]:
res3[3]

[(900142, 900212, 100700853, 900249),
 (231, 100700841, 900236, 900255),
 (900256, 22010118, 205802, 900234),
 (900225, 900259, 900268, 900269),
 (900142, 900244, 900212, 100700853),
 (900142, 900212, 900244, 100700853),
 (900256, 22010118, 900234, 900215),
 (100700834, 100700804, 900265, 209103)]

In [172]:
son_candidates = list(set(res1[3] + res2[3] + res3[3]))
son_candidates

[(22010087, 22010088, 22010094, 22010095),
 (900256, 22010118, 205802, 900234),
 (100700839, 900142, 900212, 900244),
 (900142, 900212, 900244, 100700853),
 (900142, 900212, 100700853, 900152),
 (900142, 900212, 100700853, 900249),
 (900256, 900234, 142, 900215),
 (900101, 231, 100700841, 900236),
 (900142, 900244, 900212, 100700853),
 (900225, 900259, 900268, 900269),
 (100700834, 100700804, 900265, 209103),
 (231, 100700841, 900236, 900255),
 (900256, 22010118, 900234, 900215)]

In [175]:
def check_4_tuples_son(x):
    res = []
    four_members_in_x = itertools.combinations(set(x[1]) , 4)
    for item in four_members_in_x:
        if(item in son_candidates):
            res.append((item , 1))
    return res

In [178]:
support_threshold = 1000

In [179]:
four_count_son = traffic_rdd.map(lambda x: check_4_tuples_son(x)).flatMap(lambda x: x)
four_frequnet_by_son = four_count_son.reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] > support_threshold).collect()

In [180]:
four_frequnet_by_son

[((900142, 900212, 100700853, 900249), 1012),
 ((231, 100700841, 900236, 900255), 1197),
 ((900256, 900234, 142, 900215), 1033),
 ((900225, 900259, 900268, 900269), 1198),
 ((900142, 900244, 900212, 100700853), 1287),
 ((900142, 900212, 100700853, 900152), 1051),
 ((900142, 900212, 900244, 100700853), 1372),
 ((900101, 231, 100700841, 900236), 1028),
 ((900256, 22010118, 900234, 900215), 1276),
 ((100700834, 100700804, 900265, 209103), 1263)]